In [1]:
import sddk
import pandas as pd
import tabulate
pd.options.display.max_columns = 1000 # to see all columns
import json
import re
import geopandas as gpd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import pyproj
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import pickle

import nltk
from nltk.util import bigrams

In [2]:
LIST = gpd.read_parquet("../data/large_data/LIST_contexts.parquet") # point out to your local version...

In [3]:
LIST[["not_before", "not_after"]]

,not_before,not_after
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
525865,171.0,270.0
525866,171.0,270.0
525867,101.0,300.0
525868,101.0,200.0


In [4]:
(LIST["not_before"].notnull() | LIST["not_after"].notnull()).sum()

206570

In [5]:
# remove?
EDH_overlap_all = LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].notnull())]
EDH_overlap = EDH_overlap_all[~EDH_overlap_all["type_of_inscription_clean"].str.contains("NULL", na=False)]

In [6]:
EDH_overlap

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,urban_context,urban_context_city,urban_context_pop_est
445463,445464,EDCS-24900077,HD056163,https://www.trismegistos.org/text/177366,570485,Q(uinto) Caecilio C(ai) f(ilio) Metelo / imper...,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metello imperatori I...,Q CAECILIO C F METELO / IMPERATORI ITALICI / Q...,Achaia,Agia Triada / Merbaka / Midea,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-11,None,honorific inscription,no image,NaN,None,\n Quinto Caecilio Cai filio Metelo imperatori...,None,None,1000,69,[],honorific inscription,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Midea,False,Pelopónissos,False,Midhéa,False,NULL,False,L,None,-68 to -68,-68.0,-68.0,22.841200,37.649800,True,POINT (22.841 37.650),True,rural,None,NaN
445464,445465,EDCS-03700724,HD052964,https://www.trismegistos.org/text/121715,531064,Fortissimo et piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et Piissimo Caesari domino nostro G...,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,Achaia,Agios Athanasios / Photike,tituli honorarii,"Augusti/Augustae, ordo equester, tria nomina",litterae erasae,Augusti/Augustae; litterae erasae; ordo eque...,http://db.edcs.eu/epigr/partner.php?s_language...,2014-09-16,3-5.3 cm,honorific inscription,checked with photo,NaN,Fragma Kalama,\n Fortissimo et piissimo Caesari domino nostr...,57,None,1000,69,"[{'persname': {'name': [{'@type': 'nomen', '#t...",honorific inscription,False,99.0,67.0,67.0,NULL,statue base,False,Epirus,False,Greece,False,Photike,False,Ípeiros,False,Paramythía,False,{Agios Athanasios},False,L,None,309 to 313,313.0,309.0,20.766800,39.451200,True,POINT (20.767 39.451),True,rural,None,NaN
445465,445466,EDCS-13800065,HD017714,https://www.trismegistos.org/text/177100,570049,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei quei Aegei negotiantur P Rutilium P f...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,ITALICEI / QVEI AEGEI NEGOTIANTVR / P RVTILIVM...,Achaia,Aigio / Egio / Aiyion / Aegeum,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-03-29,3.5-3.7 cm,votive inscription,checked with photo,NaN,None,\n Italicei quei Aegei negotiantur Publium Rut...,257,None,1000,372,[],votive inscription,False,58.0,61.0,16.0,NULL,tabula,False,Achaia,False,Greece

Now we can focus on some EDCS attributes (i.e. firt 28 columns) which might be good predictors of `type_of_inscription_clean` in EDH. First, look at `status_list`:

# Applying the model

In [7]:
# load the model and vectorizer back
classifier = pickle.load(open('../data/large_data/et_model_v2.0.sav', 'rb'))
vectorizer = pickle.load(open('../data/large_data/et_vectorizer_v2.0.sav','rb'))

In [8]:
# we will apply the model to inscriptions which are only in EDCS
EDCS_unique = LIST[(LIST["EDH-ID"].isnull()) & (LIST["EDCS-ID"].notnull())]
EDCS_unique.head(5)

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,urban_context,urban_context_city,urban_context_pop_est
0,1,EDCS-31400030,None,None,None,None,Leius,Leius,None,Leius,Leius,None,Achaia,?,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,NaN,NaN,False,None,False,None,None,NaN
1,2,EDCS-55701593,None,None,None,None,Cn(aeus) Atei(us),Cn Atei,None,Cnaeus Ateius,Cnaeus Ateius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.507 39.476),True,rural,None,NaN
2,3,EDCS-55701594,None,None,None,None,Avill(ius),Avill,None,Avillius,Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.507 39.476),True,rural,None,NaN
3,4,EDCS-55701595,None,None,None,None,L(ucius) Av(illius),L Av,None,Lucius Avillius,Lucius Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.507 39.476),True,rural,None,NaN
4,5,EDCS-55701596,None,None,None,None,Camurius,Camurius,None,Camurius,Camurius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.507 39.476),True,rural,None,NaN


In [9]:
# first create bigrams
def get_bigrams_underscore(inscr_text):
    try:
        inscr_bigrams = [" ".join(el) for el in list(bigrams(inscr_text.split()))]
        inscr_bigrams_ = [bigram.replace(" ", "_") for bigram in inscr_bigrams]
    except:
        inscr_bigrams_ = []
    return inscr_bigrams_

EDCS_unique["bigrams"] = EDCS_unique["clean_text_interpretive_word"].apply(get_bigrams_underscore)

In [10]:
EDCS_unique["status_list"] = EDCS_unique["status"].fillna(value="").apply(lambda x: x.split(";"))
EDCS_unique["material_clean"].replace("NULL", inplace=True)

In [11]:
def combine_status_list_and_bigrams(el_status, el_bigrams, el_material):
    # preprocess status:
    try:
        el_status = eval(el_status)
    except:
        pass
    if isinstance(el_status, list):
        new_el_status = el_status
    elif isinstance(el_status, str):
        new_el_status = [el_status]
    else:
        new_el_status = []
    new_el_status = [el.strip().replace(" ", "_") for el in new_el_status]
    # preprocess material:
    if el_material is None:
        el_material = []
    else:
        el_material = el_material.partition(": ")[0]  #
        el_material = el_material.split(", ")
        el_material = [el.replace(" ", "_").replace("?", "") for el in el_material]
    # combine status, bigrams and material
    new_el = new_el_status + el_bigrams + el_material
    new_el = " ".join(new_el)
    return new_el, new_el_status, el_material

In [12]:
# we need to extract all predictors from the dataframe
# into `status_bigrams_list_apply` (list of lists)
status_bigrams_list_apply = []
status_list = []
material_list = []
for el_status, el_bigrams, el_material in zip(EDCS_unique["status_list"].tolist(), EDCS_unique["bigrams"].tolist(), EDCS_unique["material_clean"].tolist()):
    new_el, new_status, el_material = combine_status_list_and_bigrams(el_status, el_bigrams, el_material)
    status_bigrams_list_apply.append(new_el)
    status_list.append(new_status)
    material_list.extend(el_material)

In [13]:
X_applied = vectorizer.transform(status_bigrams_list_apply)

In [14]:
prediction = classifier.predict(X_applied)
prediction[:10]

array(['identification inscription', 'owner/artist inscription',
       'owner/artist inscription', 'owner/artist inscription',
       'owner/artist inscription', 'owner/artist inscription',
       'votive inscription', 'identification inscription',
       'honorific inscription', 'building/dedicatory inscription'],
      dtype='<U31')

In [15]:
classifier.classes_

array(['acclamation', 'adnuntiatio', 'assignation inscription',
       'boundary inscription', 'building/dedicatory inscription',
       'calendar', 'defixio', 'elogium', 'epitaph',
       'honorific inscription', 'identification inscription', 'label',
       'letter', 'list', 'mile-/leaguestone', 'military diploma',
       'owner/artist inscription', 'prayer', 'private legal inscription',
       'public legal inscription', 'seat inscription',
       'votive inscription'], dtype='<U31')

In [16]:
probs = [np.max(pred) for pred in classifier.predict_proba(X_applied)]
probs[:10]

[0.6666666666666659,
 0.9031917526917528,
 0.8401048945136412,
 0.9031917526917528,
 0.8401048945136412,
 0.9031917526917528,
 0.36523809523809525,
 0.6666666666666659,
 0.62,
 0.3782555555555555]

In [17]:
prediction_probs = [(classifier.classes_[np.argmax(pred)], pred[np.argmax(pred)]) for pred in classifier.predict_proba(X_applied)]
prediction_probs[:10]

[('identification inscription', 0.6666666666666659),
 ('owner/artist inscription', 0.9031917526917528),
 ('owner/artist inscription', 0.8401048945136413),
 ('owner/artist inscription', 0.9031917526917528),
 ('owner/artist inscription', 0.8401048945136413),
 ('owner/artist inscription', 0.9031917526917528),
 ('votive inscription', 0.36523809523809525),
 ('identification inscription', 0.6666666666666659),
 ('honorific inscription', 0.62),
 ('building/dedicatory inscription', 0.3782555555555555)]

In [18]:
len(prediction_probs)

445463

In [19]:
for threshold in [thres / 100 for thres in range(40, 100, 5)]:
    above_threshold = [tup for tup in prediction_probs if tup[1] >= threshold]
    proportion = len(above_threshold) / len(prediction_probs)
    print("prob. threshold: {0}, proportion of inscr.: {1}".format(threshold, proportion))

prob. threshold: 0.4, proportion of inscr.: 0.9789567259233651
prob. threshold: 0.45, proportion of inscr.: 0.9658804434936235
prob. threshold: 0.5, proportion of inscr.: 0.942558641233952
prob. threshold: 0.55, proportion of inscr.: 0.9065332025330948
prob. threshold: 0.6, proportion of inscr.: 0.8807173659765233
prob. threshold: 0.65, proportion of inscr.: 0.8624554676819399
prob. threshold: 0.7, proportion of inscr.: 0.6787634438774938
prob. threshold: 0.75, proportion of inscr.: 0.6562295858466359
prob. threshold: 0.8, proportion of inscr.: 0.590996334151209
prob. threshold: 0.85, proportion of inscr.: 0.5466649306451938
prob. threshold: 0.9, proportion of inscr.: 0.5188713765228538
prob. threshold: 0.95, proportion of inscr.: 0.468119686707987


In [20]:
threshold_results = []
for threshold in [thres / 100 for thres in range(30, 100, 2)]:
    above_threshold = [tup for tup in prediction_probs if tup[1] >= threshold]
    proportion = len(above_threshold) / len(prediction_probs)
    threshold_results.append({"threshold (=/>)" : threshold, "proportion" : np.round(proportion, 2), "N" : len(above_threshold)})
model_results_df = pd.DataFrame(threshold_results)
model_results_df

,threshold (=/>),proportion,N
0,0.30,0.99,442779
1,0.32,0.99,442028
2,0.34,0.99,441065
3,0.36,0.99,439976
4,0.38,0.98,437708
5,0.40,0.98,436089
6,0.42,0.97,433818
7,0.44,0.97,431718
8,0.46,0.96,428311
9,0.48,0.95,424488


In [21]:
#print(model_results_df.to_latex(index=False))

In [22]:
#model_results_df.to_csv("../data/classifier_EDCS_results.csv")

# Combining the data back

In [23]:
EDCS_unique["type_of_inscription_auto"] = prediction
EDCS_unique["type_of_inscription_auto_prob"] = probs

In [24]:
EDCS_unique.head(5)

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,urban_context,urban_context_city,urban_context_pop_est,bigrams,status_list,type_of_inscription_auto,type_of_inscription_auto_prob
0,1,EDCS-31400030,None,None,None,None,Leius,Leius,None,Leius,Leius,None,Achaia,?,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,NaN,NaN,False,None,False,None,None,NaN,[],[],identification inscription,0.666667
1,2,EDCS-55701593,None,None,None,None,Cn(aeus) Atei(us),Cn Atei,None,Cnaeus Ateius,Cnaeus Ateius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.507 39.476),True,rural,None,NaN,[Cnaeus_Ateius],"[praenomen et nomen, sigilla impressa, tit...",owner/artist inscription,0.903192
2,3,EDCS-55701594,None,None,None,None,Avill(ius),Avill,None,Avillius,Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.507 39.476),True,rural,None,NaN,[],"[nomen singulare, sigilla impressa, tituli...",owner/artist inscription,0.840105
3,4,EDCS-55701595,None,None,None,None,L(ucius) Av(illius),L Av,None,Lucius Avillius,Lucius Avillius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"praenomen et nomen, viri",sigilla impressa,praenomen et nomen; sigilla impressa; tituli...,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.507 39.476),True,rural,None,NaN,[Lucius_Avillius],"[praenomen et nomen, sigilla impressa, tit...",owner/artist inscription,0.903192
4,5,EDCS-55701596,None,None,None,None,Camurius,Camurius,None,Camurius,Camurius,None,Achaia,Agios Donatos / Photike,tituli fabricationis,"nomen singulare, viri",sigilla impressa,nomen singulare; sigilla impressa; tituli fa...,None,None,None,None,None,NaN,None,None,None,None,None,None,[],None,False,NaN,NaN,NaN,None,None,False,None,False,None,False,None,False,None,False,None,False,None,False,L,None,None,NaN,NaN,20.506908,39.475976,False,POINT (20.507 39.476),True,rural,None,NaN,[],"[nomen singulare, sigilla impressa, tituli...",owner/artist inscription,0.840105


In [25]:
# prepare last part of the data - inscriptions we have only in EDH
# in this case, we can also use the information we already have
EDH_unique = LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].isnull())]
EDH_unique["type_of_inscription_auto"] = EDH_unique["type_of_inscription_clean"]
EDH_unique["type_of_inscription_auto_prob"] = [1] * len(EDH_unique)

In [26]:
EDH_overlap_all = LIST[(LIST["EDH-ID"].notnull()) & (LIST["EDCS-ID"].notnull())]
EDH_overlap_all["type_of_inscription_auto"] = EDH_overlap_all["type_of_inscription_clean"]
EDH_overlap_all["type_of_inscription_auto_prob"] = [1] * len(EDH_overlap_all)

In [27]:
print("the dataset we used for training and testing (+NULLs): ", EDH_overlap_all.shape)
print("inscriptions in EDH only: ", EDH_unique.shape)
print("inscriptions in EDCS only, we wanted to classify: ", EDCS_unique.shape)

the dataset we used for training and testing (+NULLs):  (77091, 66)
inscriptions in EDH only:  (3316, 66)
inscriptions in EDCS only, we wanted to classify:  (445463, 68)


In [28]:
# combine together
LIST_autotypes = pd.concat([EDH_overlap_all, EDH_unique, EDCS_unique])
LIST_autotypes.head()

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,urban_context,urban_context_city,urban_context_pop_est,type_of_inscription_auto,type_of_inscription_auto_prob,bigrams,status_list
445463,445464,EDCS-24900077,HD056163,https://www.trismegistos.org/text/177366,570485,Q(uinto) Caecilio C(ai) f(ilio) Metelo / imper...,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metello imperatori I...,Q CAECILIO C F METELO / IMPERATORI ITALICI / Q...,Achaia,Agia Triada / Merbaka / Midea,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-11,None,honorific inscription,no image,NaN,None,\n Quinto Caecilio Cai filio Metelo imperatori...,None,None,1000,69,[],honorific inscription,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Midea,False,Pelopónissos,False,Midhéa,False,NULL,False,L,None,-68 to -68,-68.0,-68.0,22.841200,37.649800,True,POINT (22.841 37.650),True,rural,None,NaN,honorific inscription,1.0,NaN,NaN
445464,445465,EDCS-03700724,HD052964,https://www.trismegistos.org/text/121715,531064,Fortissimo et piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et Piissimo Caesari domino nostro G...,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,Achaia,Agios Athanasios / Photike,tituli honorarii,"Augusti/Augustae, ordo equester, tria nomina",litterae erasae,Augusti/Augustae; litterae erasae; ordo eque...,http://db.edcs.eu/epigr/partner.php?s_language...,2014-09-16,3-5.3 cm,honorific inscription,checked with photo,NaN,Fragma Kalama,\n Fortissimo et piissimo Caesari domino nostr...,57,None,1000,69,"[{'persname': {'name': [{'@type': 'nomen', '#t...",honorific inscription,False,99.0,67.0,67.0,NULL,statue base,False,Epirus,False,Greece,False,Photike,False,Ípeiros,False,Paramythía,False,{Agios Athanasios},False,L,None,309 to 313,313.0,309.0,20.766800,39.451200,True,POINT (20.767 39.451),True,rural,None,NaN,honorific inscription,1.0,NaN,NaN
445465,445466,EDCS-13800065,HD017714,https://www.trismegistos.org/text/177100,570049,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei quei Aegei negotiantur P Rutilium P f...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,ITALICEI / QVEI AEGEI NEGOTIANTVR / P RVTILIVM...,Achaia,Aigio / Egio / Aiyion / Aegeum,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-03-29,3.5-3.7 cm,votive inscription,checked with photo,NaN,None,\n It

In [29]:
LIST_autotypes.drop(["bigrams", "status_list"], axis=1, inplace=True)

In [30]:
LIST_autotypes.shape

(525870, 66)

In [31]:
LIST_autotypes.head(5)

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,geotemporal?,geometry,withinRE?,urban_context,urban_context_city,urban_context_pop_est,type_of_inscription_auto,type_of_inscription_auto_prob
445463,445464,EDCS-24900077,HD056163,https://www.trismegistos.org/text/177366,570485,Q(uinto) Caecilio C(ai) f(ilio) Metelo / imper...,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metello imperatori I...,Q CAECILIO C F METELO / IMPERATORI ITALICI / Q...,Achaia,Agia Triada / Merbaka / Midea,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-11,None,honorific inscription,no image,NaN,None,\n Quinto Caecilio Cai filio Metelo imperatori...,None,None,1000,69,[],honorific inscription,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Midea,False,Pelopónissos,False,Midhéa,False,NULL,False,L,None,-68 to -68,-68.0,-68.0,22.841200,37.649800,True,POINT (22.841 37.650),True,rural,None,NaN,honorific inscription,1.0
445464,445465,EDCS-03700724,HD052964,https://www.trismegistos.org/text/121715,531064,Fortissimo et piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et Piissimo Caesari domino nostro G...,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,Achaia,Agios Athanasios / Photike,tituli honorarii,"Augusti/Augustae, ordo equester, tria nomina",litterae erasae,Augusti/Augustae; litterae erasae; ordo eque...,http://db.edcs.eu/epigr/partner.php?s_language...,2014-09-16,3-5.3 cm,honorific inscription,checked with photo,NaN,Fragma Kalama,\n Fortissimo et piissimo Caesari domino nostr...,57,None,1000,69,"[{'persname': {'name': [{'@type': 'nomen', '#t...",honorific inscription,False,99.0,67.0,67.0,NULL,statue base,False,Epirus,False,Greece,False,Photike,False,Ípeiros,False,Paramythía,False,{Agios Athanasios},False,L,None,309 to 313,313.0,309.0,20.766800,39.451200,True,POINT (20.767 39.451),True,rural,None,NaN,honorific inscription,1.0
445465,445466,EDCS-13800065,HD017714,https://www.trismegistos.org/text/177100,570049,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei quei Aegei negotiantur P Rutilium P f...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,ITALICEI / QVEI AEGEI NEGOTIANTVR / P RVTILIVM...,Achaia,Aigio / Egio / Aiyion / Aegeum,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-03-29,3.5-3.7 cm,votive inscription,checked with photo,NaN,None,\n Italicei quei Aegei negotiantur Publiu

In [32]:
len(LIST_autotypes[LIST_autotypes["type_of_inscription_auto_prob"]>=0.6])

472734

In [33]:
len(LIST_autotypes[LIST_autotypes["type_of_inscription_auto"]=="NULL"])

20388

In [34]:
def apply_threshold(inscr_type, prob):
    if (prob < 0.6) or (inscr_type == "NULL"):
        return None
    else:
        return inscr_type
LIST_autotypes["type_of_inscription_auto"] =  LIST_autotypes.apply(lambda row: apply_threshold(row["type_of_inscription_auto"], row["type_of_inscription_auto_prob"]), axis=1)

In [35]:
def prob_to_zero(inscr_type, prob):
    if inscr_type == None:
        return 0
    else:
        return prob
LIST_autotypes["type_of_inscription_auto_prob"] =  LIST_autotypes.apply(lambda row: prob_to_zero(row["type_of_inscription_auto"], row["type_of_inscription_auto_prob"]), axis=1)

In [36]:
len(LIST_autotypes[LIST_autotypes["type_of_inscription_auto"].notnull()])

451839

# Save

In [37]:
LIST_autotypes.to_parquet("../data/large_data/LIST_v0-4.parquet")

In [38]:
LIST_autotypes.to_file("../data/large_data/LIST_v0-4.geojson", driver="GeoJSON")